In [137]:
import pandas as pd
import sklearn
from sklearn import preprocessing
import datetime
!pip install boruta
from boruta import BorutaPy

You are using pip version 10.0.1, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [103]:
# Gradient Boosting Machine
gmb = GradientBoostingClassifier()
le_day_of_week = preprocessing.LabelEncoder()
le_week_status = preprocessing.LabelEncoder()

def secondsDay(data_frame):
    hour = data_frame.date.dt.hour
    minute = data_frame.date.dt.minute
    seconds = data_frame.date.dt.second
    return hour * 3600 + minute * 60 + seconds
    
def weekStatus(data_frame, label_encoder):
    week_status_DF = pd.DataFrame(columns=['week_status'])
    train_date_DF = pd.DataFrame(columns=['date'])
    train_date_DF = data_frame['date']    
    for date in train_date_DF.items():
        day = date[1].weekday_name
        if day == 'Saturday' or day == 'Sunday':
            week_status_DF =  week_status_DF.append({'week_status': 'Weekend'}, ignore_index=True)
        else:
            week_status_DF =  week_status_DF.append({'week_status': 'Weekday'}, ignore_index=True)
    
    weekstatus_labels = label_encoder.fit_transform(week_status_DF.values.ravel())
    return weekstatus_labels

def dayOfWeek(data_frame, label_encoder):
    day_of_week_label = label_encoder.fit_transform(data_frame.date.dt.weekday_name.values.ravel())
    return day_of_week_label

In [104]:
# Pre-process data
train_data = pd.read_csv('s3://ml-assignment2-data/UCI-electricity/UCI_data.csv', parse_dates=True)

# Generate timeseries eatures for complete dataset
train_data['date'] = train_data['date'].astype('datetime64[ns]')

# Feature Generation for Timeseries Data
# Generate Number of Seconds from Midnight (NSM), 
nsm_series = secondsDay(train_data)
nsm_DF = nsm_series.to_frame()

# Week Status of the data, weekend or weekday
week_status_DF = pd.DataFrame(data=weekStatus(train_data, le_week_status), columns=['week_status'])

# DayOfTheWeek, for each data row
day_of_week_DF = pd.DataFrame(data=dayOfWeek(train_data, le_day_of_week), columns=['day_of_week'])

train_data['nsm'] = nsm_DF['date']
train_data['week_status'] = week_status_DF['week_status']
train_data['day_of_week'] = day_of_week_DF['day_of_week']
train_data =  train_data.drop(columns=['date'])

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:17: FutureWarning: `weekday_name` is deprecated and will be removed in a future version. Use `day_name` instead


In [105]:
# SKLEARN to create data partition function, 75% Training, 25% Testing. 
x_train_DF = train_data.drop(columns=['TARGET_energy'])
y_train_DF = pd.DataFrame(train_data['TARGET_energy'], columns=['TARGET_energy'])
print(type(y_train_DF))
print(type(x_train_DF))
x_train_values = x_train_DF.to_numpy().astype('float64')
y_train_values = y_train_DF.to_numpy().astype('float64')

X_train, X_test, y_train, y_test = train_test_split(x_train_values, y_train_values, test_size=.25, random_state=42)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [106]:
independent_scaler = preprocessing.StandardScaler()
target_scaler = preprocessing.StandardScaler()

x_train_scaled = independent_scaler.fit_transform(X_train)
y_train_scaled = target_scaler.fit_transform(y_train.reshape(-1, 1))

X_train_scaled_DF = pd.DataFrame(x_train_scaled, columns=x_train_DF.columns)
Y_train_scaled_DF = pd.DataFrame(y_train_scaled, columns=y_train_DF.columns)

In [141]:
# Import Model Performance metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# Model & Feature Selection
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFECV
from sklearn.model_selection import KFold

# Import Models
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble  import RandomForestRegressor
from sklearn.svm import SVR

In [115]:
rf = RandomForestRegressor(n_estimators=20, criterion='mse')

feat_selector = RFECV(estimator=rf, step=1, cv=KFold(2))
feat_selector.fit(X_train_scaled_DF.to_numpy(), Y_train_scaled_DF.to_numpy().ravel())
feat_selector.support_


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
       False,  True])

In [116]:
feat_selector.ranking_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 1, 3, 1])

In [138]:
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'brier_score_loss',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'mutual_info_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'v_measure_score']

In [ ]:
import multiprocessing as mp

models= []
gmb = GradientBoostingRegressor(random_state=4)
lm = LinearRegression()
rfg = RandomForestRegressor()
svr = SVR()

models.append(('GMB',gmb))
models.append(('LM',lm))
models.append(('RFG',rfg))
models.append(('SVC',svr))

scoring = 'r2'

results= []
names = []

for name, model in models:
    kfold = KFold(n_splits=10, random_state=4)
    cv_results = cross_val_score(model, X_train_scaled_DF.to_numpy(), Y_train_scaled_DF.to_numpy().ravel(), cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

GMB: 0.307757 (0.016404)
LM: 0.158134 (0.014497)


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.",

RFG: 0.536226 (0.047532)


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/ec2-